In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/sql_prompts_with_DSPy

In [0]:
%run ../helper/GradingModule

In [0]:
module = Module(databricks.sdk.WorkspaceClient())

In [0]:
question_with_no_answer = "Q6_no_answer"
notebook_url = "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-X-no-answer-q6"
human_grades = [15,15,15,12,15,0]
module_to_check = "SQL"

In [0]:
candidate_dict = {
  "X": {
    f"SQL_{question_with_no_answer}": notebook_url
  }
}

human_graded_dict = {
  "X": {
    f"SQL_{question_with_no_answer}": human_grades
  }
}

llm_models = {
  # 'llm-405B': 'databricks/databricks-meta-llama-3-1-405b-instruct',
  'llm-70B': 'databricks/databricks-meta-llama-3-3-70b-instruct'
  # 'claude': "databricks/databricks-claude-3-7-sonnet"
}

In [0]:
print(candidate_dict)
print(human_graded_dict)

In [0]:
def evaluate_scores(predictions, human_grading, candidate, section):
  mismatch_dict = {}

  scores_list = predictions['score'].tolist()
  code_snippet_list = predictions['code_snippet'].tolist()
  reasoning_list = predictions['chain_of_thought_reasoning'].tolist()

  for i in range(0, len(human_grading)):
    if scores_list[i]!= human_grading[i]:
      mismatch_dict[f"{candidate}_{section}_q{i+1}"] = {
        'predicted_score': scores_list[i],
        'human_score': human_grading[i],
        'code_snippet': code_snippet_list[i],
        'reasoning': reasoning_list[i]
      }
  if not mismatch_dict:
    return None
  
  print(mismatch_dict)
  return mismatch_dict

In [0]:
import time 

test_results = {}
model_dict = {} # To store end results for each model

# Do multiple runs to check consistency
for i in range(0,20):  

  # iterate through multiple llms 
  for k,v in llm_models.items():
    start_time = time.time()

    # Iterate through multiple candidates 
    for candidate, values in candidate_dict.items():
      
      # Store answers for a candidate
      answers_dict = {}

      # Store results which are not matching 
      candidate_mismatch = {}

      # Iterate through multiple notebooks of a candidate
      for section, context_path in values.items():

        # Create a new instance of llm module to empty out the cache 
        lm = dspy.LM(llm_models[k], cache=False, temperature=0)
        dspy.configure(lm=lm)
        
        # Set the module dictionary to be used. 
        # A module dictionary will have prompts for each question in a module 
        module.set_module_dict(module_to_check)
        
        # Get results for the section 
        results = module.get_error_and_answer_dict(context_path)
        
        # Add info for candidate, section and context in the answers 
        results['answers_dict']['candidate'] = candidate
        results['answers_dict']['section'] = section
        results['answers_dict']['context'] = results['context'] 
        
        # Get results of a section and evaluate  
        answers_dict[f'{candidate}_{section}'] = results['answers_dict']
        answers_df = pd.concat(answers_dict.values(), ignore_index=True)
        # answers_df.display()
        candidate_mismatch[f'{candidate}_{section}'] = evaluate_scores(answers_df, human_graded_dict[candidate][section], candidate, section)

        # End time calculation 
        end_time = time.time()
        execution_time = end_time - start_time
        print(f" time taken by model {k} for execution of section {section} is {execution_time} seconds")
        

    model_dict[f'{k}_round_{i+1}'] = {'answers_dict': answers_dict, 'execution_time': execution_time, 'lm':lm, 'candidate_mismatch': candidate_mismatch}
  print(f" run completed {i+1} \n")
  

In [0]:
mismatch_dict = {k:v['candidate_mismatch'] for k, v in model_dict.items()}
mismatch_dict

In [0]:
df = pd.DataFrame()
for k, v in mismatch_dict.items(): 
  temp_df = pd.concat(model_dict[k]['answers_dict'].values(), ignore_index=True)
  df = pd.concat([df, temp_df], ignore_index=True)
df.display()

In [0]:
f"users.abhay_jalisatgi.test_{question_with_no_answer}_{module_to_check}"

In [0]:
spark.createDataFrame(df).write.mode("overwrite").saveAsTable(f"users.abhay_jalisatgi.test_{question_with_no_answer}_{module_to_check}")